In [1]:
# default_exp ipython_magic

In [2]:
#export

from IPython.core.magic import register_cell_magic
import util

_node_cache = ''
_notebook_name = None

@register_cell_magic
def node(arg, cell, test=False):
    global _node_cache
    global _notebook_name

    ipython_node_path = '_ipython_node.ts'
    
    def split_imports(code): return util.list_partition(code.splitlines(), lambda line: line.startswith('import '))
    cache_imports, cache_exports = split_imports(_node_cache)
    cell_imports, cell_exports = split_imports(cell)
    cell_exports = '\n'.join(cell_exports).strip('\n').split('\n')

    if arg == 'run':
        ts_imports = cache_imports
        ipython_imports = cache_imports + cell_imports

        ts_exports = cache_exports
        ipython_exports = cache_exports \
            + ([] if cell_exports == [''] 
            else (['', '// cell', 'var run = async (arg: string) => {'] + cell_exports + ['}', f'await run("{arg}")']))
    elif arg == 'export':
        ts_imports = cache_imports + cell_imports
        ipython_imports = ts_imports

        ts_exports = cache_exports + ([] if cell_exports == [''] else (['', '// cell'] + cell_exports))
        ipython_exports = ts_exports

    def join_imports(imports, exports): 
        return ''.join([
            '\n'.join(list(dict.fromkeys(imports))),
            '' if len(imports) == 0 or len(exports) == 0 else '\n\n\n',
            '\n'.join(exports).strip('\n') + '\n'
        ])
    new_code_ts = join_imports(ts_imports, ts_exports)
    new_code_ipython = join_imports(ipython_imports, ipython_exports)

    util.write_file(ipython_node_path, new_code_ipython)

    result = util.run_node(ipython_node_path).splitlines() if arg == 'run' else []

    _node_cache = new_code_ts

    if test:
        return result, new_code_ts, new_code_ipython
    else:
        if _notebook_name is None:
            _notebook_name = util.get_notebook_name()

        ts_node_path = f'{_notebook_name}.ts'

        util.write_file(ts_node_path, new_code_ts)
        return result if result != [] else ts_node_path


In [3]:
_node_cache = ''

def test_cell_node(arg, cell, expected_ts, expected_ipython, expected_result):
    result, new_code_ts, new_code_ipython = node(arg, cell, test=True)
    if new_code_ts != expected_ts or new_code_ipython != expected_ipython or result != expected_result:
        raise Exception('\n' + '\n'.join(['\n|||arg:', arg, '\n|||cell:', cell, 
                        '\n|||new_code_ts:', new_code_ts, '\n|||expected_ts:', expected_ts, 
                        '\n|||new_code_ipython:', new_code_ipython, '\n|||expected_ipython:', expected_ipython, 
                        '\n|||result:', str(result), '\n|||expected_result:', str(expected_result)]))
    else:
        print('arg:', arg, 'result:', expected_result)

test_cell_node(
    'export',
    'import * as os from "os"',
    'import * as os from "os"\n',
    'import * as os from "os"\n',
    [])

test_cell_node(
    'export',
    'import * as os from "os"\nvar a = os.EOL',
    'import * as os from "os"\n\n\n// cell\nvar a = os.EOL\n',
    'import * as os from "os"\n\n\n// cell\nvar a = os.EOL\n',
    [])

test_cell_node(
    'export',
    'import * as fs from "fs"\nvar b = fs.constants.F_OK',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n',
    [])

test_cell_node(
    'run',
    'console.log(a)',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n\n// cell\nvar run = async (arg: string) => {\nconsole.log(a)\n}\nawait run("run")\n',
    [])

test_cell_node(
    'run',
    'console.log(b)',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n\n// cell\nvar run = async (arg: string) => {\nconsole.log(b)\n}\nawait run("run")\n',
    ['0'])

test_cell_node(
    'export',
    'import * as fs from "fs"\nvar c = fs.constants.X_OK',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n\n// cell\nvar c = fs.constants.X_OK\n',
    'import * as os from "os"\nimport * as fs from "fs"\n\n\n// cell\nvar a = os.EOL\n\n// cell\nvar b = fs.constants.F_OK\n\n// cell\nvar c = fs.constants.X_OK\n',
    [])

_node_cache = ''

arg: export result: []
arg: export result: []
arg: export result: []
arg: run result: []
arg: run result: ['0']
arg: export result: []


In [4]:
#export

import shutil
import os
from time import sleep
from IPython.core.magic import register_cell_magic

def _get_spiral_cache_empty(): return {'spi': '', 'spir': '', 'run': '', '': ''}
_spiral_cache = _get_spiral_cache_empty()

@register_cell_magic
def spiral(arg, cell, test=False):
    global _spiral_cache
    global _notebook_name

    last_spiral_cache = _spiral_cache

    args = arg.split(' ')
    def get_arg(i, d=''): return next(iter(args[i:i+1]), d)
    arg = args[0]

    def split_imports(code): return util.list_partition(code.splitlines(), lambda line: line.startswith('open '))
    cache_imports, cache_exports = split_imports(_spiral_cache[arg])
    
    cell_imports, cell_exports = split_imports(cell)
    cell_exports = '\n'.join(cell_exports).strip('\n').split('\n')

    if arg == 'run':
        cell_imports = ['open util', 'open console_fsx'] + cell_imports
        cell_exports = ['inl main () : () = '] + list(map(lambda l: f'    {l}', cell_exports))

    spi_imports = cache_imports + cell_imports
    spi_exports = cache_exports + ([] if cell_exports == [''] else (['', '// cell'] + cell_exports))

    def join_imports(imports, exports): 
        return ''.join([
            '\n'.join(list(dict.fromkeys(imports))),
            '' if len(imports) == 0 or len(exports) == 0 else '\n\n\n',
            '\n'.join(exports).strip('\n') + '\n'
        ])
    new_code_spi = join_imports(spi_imports, spi_exports)
    
    if arg in ['spi', 'spir']:
        if _notebook_name is None:
            _notebook_name = util.get_notebook_name()
        spi_path = f'{_notebook_name}.{arg}'
    else:
        spi_path = f'main.spi'
    
    _spiral_cache[arg] = new_code_spi

    if test:
        return new_code_spi
    else:
        if arg == 'run': 
            shutil.copyfile('main.spi', 'main.spi.tmp')
            shutil.copyfile('main.fsx', 'main.fsx.tmp')
            shutil.copyfile('main.py', 'main.py.tmp')
        try:
            util.write_file(spi_path, new_code_spi)

            if arg == 'run':
                sleep(0.1)
                code = '\n'.join([
                    'import * as spiral_api from "./spiral_api"',
                    f'await spiral_api.spiToFsx("{os.path.join(os.getcwd(), "main.spi")}")'
                ])
                result = util.run_node(code, timeout=int(get_arg(1, 3))).splitlines()
                sleep(0.1)
                fsx_output = util.read_file("main.fsx")
                os.rename('main.fsx', '_ipython_spi.fsx')
                
                print(f'build result: {result}')
                print(f'fsx_output: {fsx_output}')
        except Exception as e:
            print(f'ipython_magic.spiral() error. new_code_spi={new_code_spi}')
            _spiral_cache = last_spiral_cache
            raise e
        finally:
            if arg == 'run': 
                os.rename('main.spi.tmp', 'main.spi')
                os.rename('main.fsx.tmp', 'main.fsx')
                os.rename('main.py.tmp', 'main.py')

        return spi_path

@register_cell_magic
def spi(arg, cell, test=False):
    return spiral('spi', cell, test)

@register_cell_magic
def spir(arg, cell, test=False):
    return spiral('spir', cell, test)


In [5]:
_spiral_cache = _get_spiral_cache_empty()

def test_cell_spiral(arg, cell, expected_spi):
    new_code_spi = spiral(arg, cell, test=True)

    if new_code_spi != expected_spi:
        raise Exception('\n' + '\n'.join(['\n|||arg:', arg, '\n|||cell:', cell, 
                        '\n|||new_code_spi:', new_code_spi, '\n|||expected_spi:', expected_spi]))
    else:
        print('arg:', arg)

test_cell_spiral(
    'spi',
    'open a',
    'open a\n'
)

test_cell_spiral(
    'spi',
    'open a\ninl get1 () = 1i32',
    'open a\n\n\n// cell\ninl get1 () = 1i32\n'
)

test_cell_spiral(
    'spi',
    'open b\ninl get2 () = 2i32',
    'open a\nopen b\n\n\n// cell\ninl get1 () = 1i32\n\n// cell\ninl get2 () = 2i32\n'
)

test_cell_spiral(
    'run',
    'open b\ninspect 1i32',
    'open util\nopen console_fsx\nopen b\n\n\n// cell\ninl main () : () = \n    inspect 1i32\n'
)

test_cell_spiral(
    'spi',
    'open b\ninl get3 () = 3i32',
    'open a\nopen b\n\n\n// cell\ninl get1 () = 1i32\n\n// cell\ninl get2 () = 2i32\n\n// cell\ninl get3 () = 3i32\n'
)

_spiral_cache = _get_spiral_cache_empty()

<magic:f48c5c0eb5404cc68e41424f4a45ec43>


<IPython.core.display.Javascript object>

arg: spi
arg: spi
arg: spi
arg: run
arg: spi


In [6]:
%%spiral run

inspect "test"


build result: ['0 s, 52.172 ms - spiral.spiToFsx() end']
fsx_output: 
let v0 : string = "test"
printfn $"%A{v0}"



'main.spi'